### Structured Streaming - práctica


#### Consultas sobre ficheiros - escritura a disco




In [ ]:
# Extrae un mínimo de 3 ficheiros de diferentes tamaños a partir de purchases.txt para realizar este exercicio
# compras1.tsv <- os 5.000 primeiros rexistros
# compras2.tsv <- 100.000 rexistros do medio do ficheiro orixinal (desde o 100.000 ao 200.000)
# compras3.tsv <- os 20.000 últimos rexistros

# Neste exercicio traballarás en local

# Escolle un directorio de traballo
# Crea un directorio input e move o fichieor compras1.tsv dentro
# Crea un directorio store e garda dentro os ficheiros csv restantes

In [ ]:
### Neste exercicio a saída debe ser a disco
# Structured Streaming proporciona diferentes tipos de "output sinks":
# - consola
# - ficheiros: csv, json, orc, parquet  (file sink)
# - kafka
# - ...

# Escribe o resultado a 'output/resultado.csv'

In [ ]:
# File Sink:
# outputMode : soporta unicamente "append
# format: csv/parquet/json/...
# options:
## path
## header
# ...

In [ ]:
# Interesante artigo sobre Output Sinks en medium.com
# https://medium.com/expedia-group-tech/apache-spark-structured-streaming-output-sinks-3-of-6-ed3247545fbc

In [ ]:
# 1. Lanza unha consulta filtre e garde a ficheiro unicamente as vendas cun valor > 300
#    a partir dos ficheiros en 'input'

# No primeiro momento estará compras1.txt -> espera a que se execute a consulta
# Move o segundo ficheiro e observa o resultado
# Move o terceiro documento e observa o resultado

# Compara a evolución dos datos ao ir procesando os diferentes ficheiros

In [ ]:
# Importa as librarías necesarias
from pyspark.sql.types import StructType

In [ ]:
# Definimos o esquema do Dataframe
userSchema = StructType() \
    .add("data", "date") \
    .add("hora", "string") \
    .add("tenda", "string") \
    .add("categoria", "string") \
    .add("venda", "float") \
    .add("pago", "string")

#userSchema

In [ ]:
# Creamos un Dataframe para ler o fluxo de ficheiros
tsvFiles = spark \
    .readStream \
    .option("sep", "\t") \
    .schema(userSchema) \
    .csv("file:///home/hduser/code/spark/input")

In [ ]:
tsvFiles.createOrReplaceTempView('compras')
df_sql = spark.sql('SELECT * FROM compras WHERE venda > 300')

In [ ]:
query = df_sql \
    .writeStream \
    .outputMode("append") \
    .format("csv") \
    .option('header',True) \
    .option('path',"file:///home/hduser/code/spark/output/resultado.csv") \
    .option("checkpointLocation", "file:///home/hduser/code/spark/output/checkpoint/filesink_checkpoint") \
    .start()

query.awaitTermination()